<a href="https://colab.research.google.com/github/SifatAnik/Data-Science-Project/blob/main/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd
import numpy as np
import nltk
import re

from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')

from nltk.corpus import brown

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
df = pd.read_csv('/content/drive/MyDrive/DM project /amazon.csv', index_col=0,header=0,error_bad_lines=False)
df.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
df.head(3)

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,NaN,4,No issues.,23-07-2014,138,0,0,0,0,0.0,0.0
1,0mie,5,"Purchased this for my device, it worked as adv...",25-10-2013,409,0,0,0,0,0.0,0.0
2,1K3,4,it works as expected. I should have sprung for...,23-12-2012,715,0,0,0,0,0.0,0.0


We require only the reviewText, overall columns which describes the reviews, star rating of each review respectively.

In [11]:
df=df[['reviewText','overall']]

 Remove the Null, missing values and reset the 

In [12]:
df=df.dropna()
df = df.reset_index(drop=True)
df

,reviewText,overall
0,No issues.,4
1,"Purchased this for my device, it worked as adv...",5
2,it works as expected. I should have sprung for...,4
3,This think has worked out great.Had a diff. br...,5
4,"Bought it with Retail Packaging, arrived legit...",5
...,...,...
4909,I bought this Sandisk 16GB Class 10 to use wit...,1
4910,Used this for extending the capabilities of my...,5
4911,Great card that is very fast and reliable. It ...,5
4912,Good amount of space for the stuff I want to d...,5


Labelling Reviews:Now we have 4914 reviews. The reviews with star rating 4,5 are labelled as positive reviews and 1,2 are labelled as negative reviews. Remove the reviews with star rating 3 as they are considered as neutral.

In [13]:
df['overall']=df['overall'].astype(int) #convert the star_rating column to int
df=df[df['overall']!=3]
df['label']=np.where(df['overall']>=4,1,0) #1-Positve,0-Negative
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,reviewText,overall,label
0,No issues.,4,1
1,"Purchased this for my device, it worked as adv...",5,1
2,it works as expected. I should have sprung for...,4,1
3,This think has worked out great.Had a diff. br...,5,1
4,"Bought it with Retail Packaging, arrived legit...",5,1
...,...,...,...
4909,I bought this Sandisk 16GB Class 10 to use wit...,1,0
4910,Used this for extending the capabilities of my...,5,1
4911,Great card that is very fast and reliable. It ...,5,1
4912,Good amount of space for the stuff I want to d...,5,1


Number of reviews group by star rating

In [14]:
df['overall'].value_counts()

5    3921
4     527
1     244
2      80
Name: overall, dtype: int64

Now we are creating the model by considering the 2000 reviews. In the 2000 reviews 1000 are positive and 1000 are negative.

In [57]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle
data=df[df['label']==0][:300]
data=data.append(df[df['label']==1][:300])
data = data.reset_index(drop=True)
display(data['label'].value_counts())
data

0    300
1    300
Name: label, dtype: int64

,reviewText,overall,label
0,"First, it kept unmounting on the Galaxy Note 1...",1,0
1,I purchased four (4) items from amazon and whe...,1,0
2,I bougth this micro SD card after some trubles...,1,0
3,I bought this to be able to load more music in...,1,0
4,never looked to verify the capacity of this ca...,1,0
...,...,...,...
595,this card is great you can save lots of pictur...,5,1
596,These cards work well with many of our pieces ...,5,1
597,I purchased this product specifically to utili...,5,1
598,PROsIt copies large files to the card at ~10-1...,5,1


# Pre-Processing
The first step is convert the all reviews into the lower case.

In [58]:
data['pre_process'] = data['reviewText'].apply(lambda x: "".join(x.lower() for x in str(x).split()))

Remove the HTML tags and URLs from the reviews.

In [59]:
from bs4 import BeautifulSoup
data['pre_process']=data['pre_process'].apply(lambda x: BeautifulSoup(x).get_text())
import re
data['pre_process']=data['pre_process'].apply(lambda x: re.sub(r"http\S+","",  x))

In [60]:
def contractions(s):
 s = re.sub(r"won't", "will not",s)
 s = re.sub(r"would't", "would not",s)
 s = re.sub(r"could't", "could not",s)
 s = re.sub(r"\'d", " would",s)
 s = re.sub(r"can\'t", "can not",s)
 s = re.sub(r"n\'t", " not", s)
 s= re.sub(r"\'re", " are", s)
 s = re.sub(r"\'s", " is", s)
 s = re.sub(r"\'ll", " will", s)
 s = re.sub(r"\'t", " not", s)
 s = re.sub(r"\'ve", " have", s)
 s = re.sub(r"\'m", " am", s)
 return s
data['pre_process']=data['pre_process'].apply(lambda x:contractions(x))

In [61]:
import matplotlib.pyplot as plt
import os
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import joblib


In [62]:
data['pre_process']= data['pre_process'].apply(lambda x: "" .join([re.sub('[^A-Za-z]+','', x) 
 for x in nltk.word_tokenize(x)]))

Remove the extra spaces between the words

In [63]:
data['pre_process']=data['pre_process'].apply(lambda x: re.sub(' +', ' ', x))

Remove the stop words by using the NLTK package

In [64]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words()
data['pre_process']=data['pre_process'].apply(lambda x: "" "".join([x for x in x.split() if x not in stop]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Perform lemmatization using the wordnet lemmatizer

In [65]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [66]:
data

,reviewText,overall,label,pre_process
0,"First, it kept unmounting on the Galaxy Note 1...",1,0,firstitkeptunmountingonthegalaxynoteigotitrepl...
1,I purchased four (4) items from amazon and whe...,1,0,ipurchasedfouritemsfromamazonandwheniopenedthe...
2,I bougth this micro SD card after some trubles...,1,0,ibougththismicrosdcardaftersometrubleswithothe...
3,I bought this to be able to load more music in...,1,0,iboughtthistobeabletoloadmoremusicintomyverizo...
4,never looked to verify the capacity of this ca...,1,0,neverlookedtoverifythecapacityofthiscarditiscl...
...,...,...,...,...
595,this card is great you can save lots of pictur...,5,1,thiscardisgreatyoucansavelotsofpicturesorfiles...
596,These cards work well with many of our pieces ...,5,1,thesecardsworkwellwithmanyofourpiecesofelectro...
597,I purchased this product specifically to utili...,5,1,ipurchasedthisproductspecificallytoutilizeitfo...
598,PROsIt copies large files to the card at ~10-1...,5,1,prositcopieslargefilestothecardatmbsanditdoesn...


In [67]:
#idf(t) = log [ n / df(t) ] + 1 = log[ number of documents / number of documents containing the term]+1

In [68]:
from sklearn.model_selection import train_test_split

In [69]:
X_train, X_test, y_train, y_test = train_test_split(data["reviewText"], data['label'],
                                                    random_state=103)

In [70]:
print(X_train)

449    I bought these for my kids tablets and for the...
41     Within an hour of using the memory card, it fa...
22     Bought it, installed it in my Droid RazrM neve...
222    So far, so good.  It does what it's supposed t...
590    How cool it is to be able to put ALL of my mus...
                             ...                        
279    First of all Amazon has shipped the mobile ver...
241    The cars seemed great, but occasionally the ca...
198    The SD to microSD card adapter that comes with...
347    fast delivery, good speeds, and good price mak...
73     Broke after  2 weeks.....I will go back to Sam...
Name: reviewText, Length: 450, dtype: object


In [71]:
print(y_train)

449    1
41     0
22     0
222    0
590    1
      ..
279    0
241    0
198    0
347    1
73     0
Name: label, Length: 450, dtype: int64


In [72]:
print(X_train)

449    I bought these for my kids tablets and for the...
41     Within an hour of using the memory card, it fa...
22     Bought it, installed it in my Droid RazrM neve...
222    So far, so good.  It does what it's supposed t...
590    How cool it is to be able to put ALL of my mus...
                             ...                        
279    First of all Amazon has shipped the mobile ver...
241    The cars seemed great, but occasionally the ca...
198    The SD to microSD card adapter that comes with...
347    fast delivery, good speeds, and good price mak...
73     Broke after  2 weeks.....I will go back to Sam...
Name: reviewText, Length: 450, dtype: object


In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer().fit(X_train)

In [74]:
len(vect.get_feature_names())

3402

In [75]:
X_train_vectorized = vect.transform(X_train)

In [76]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

LogisticRegression()

In [77]:
predictions = model.predict(vect.transform(X_train))

In [78]:
print(predictions)

[1 0 0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 0
 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 1 0 0 1 1 1 0
 0 1 1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 0 0 1 0 1
 1 0 1 1 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1 1 0 0 0 0 1 1 0 0 1 1 1 1 0 0 1 0 0
 1 1 1 0 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 0 0 1 0
 1 1 0 1 1 0 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 0
 0 0 0 0 1 1 1 0 0 1 1 1 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 1 1 0 0 0 0 1 0 1
 1 0 1 1 1 1 0 0 0 1 1 0 1 1 1 1 0 0 0 1 0 0 1 0 1 0 1 1 1 0 1 0 1 0 0 1 1
 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 1 0 1 1 1 1 0 0 1 1 0 0 1 1 0
 0 0 0 1 0 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 0 1 0 0 0
 1 1 0 0 1 1 1 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 0 1
 1 0 0 1 0 0 0 0 1 0 0 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 0 1 0 1 1 0 0 0
 1 0 0 0 1 0]


In [79]:
#compute AUC
from sklearn.metrics import roc_auc_score
print("AUC:", roc_auc_score(y_test, predictions))

ValueError: ignored

In [ ]:
#X_train.shape
#X_train.reshape(-1,1)

In [ ]:
df.shape['overall']

In [ ]:
len(df.overall )

In [ ]:
len(df.reviewText)

In [ ]:
v = TfidfVectorizer()
v.fit(X_train)
transform_output = v.transform(X_train)

In [ ]:
#let's print the vocabulary
print(v.vocabulary_)

In [ ]:
#let's print the idf of each word:

all_feature_names = v.get_feature_names_out()

for word in all_feature_names:
    
    #let's get the index in the vocabulary
    indx = v.vocabulary_.get(word)
    
    #get the score
    idf_score = v.idf_[indx]
    
    print(f"{word} : {idf_score}")

In [ ]:
#let's print the transformed output from tf-idf
print(transform_output.toarray())